# ch4 word2vec 속도개선

word2vec은 단순 2층 신경망이라 간단하게 구현 가능. 하지만 가장 큰 문제는 말뭉치에 포함된 어휘 수가 많아지면 계산량도 커진다는 점. word2vec의 속도 개선을 위해 다음 두 가지 추가
1. Embedding이라는 새로운 계층 도입
2. 네거티브 샘플링이라는 새로운 손실함수 도입

## 4.1 개선 1 - Embedding

어휘가 100만개, 은닉충 뉴런이 100개인 cbow의 모델에서는 입력층과 출력층에 각 100만개의 뉴런이 있다고 하면
1. 입력층의 원핫 표현과 가중치 행렬 $W_{in}$ 의 곱
2. 은닉충과 가중치 행렬 $W_{out}$ 의 곱 및 Softmax 계층의 계산
의 계산에 많은 시간이 소요된다.


1. 입력층의 원핫 표현과 가중치 행렬 $W_{in}$ 의 곱
어휘 수가 많아지면 원학 표현의 벡터 크기도 커진다. 이 원핫 벡터와 가중치 행렬 $W_{in}$를 곱해야 한다.   
==>Embedding 도입
2. 은닉충과 가중치 행렬 $W_{out}$의 곱 계산량이 많다  
==>네거티브 샘플링이라는 새로운 손실 함수 도입

### Embedding 계층

In [1]:
# 그림 4-3

원핫 표현과 MMatMul 계층의 가중치 행렬을 곱하는 것은 단지 행렬의 특정 행을 추출하는 것. 따라서 원핫 표현으로의 변화와 MatMul계층의 행렬 곱은 사실상 필요가 없는 것.  
가중치 매개변수로부터 '단어 ID에 해당하는 행(벡터)'을 추출하는 계층 : Embedding 계층

### Embedding 계층 구현

In [2]:
import numpy as np
W = np.arange(21).reshape(7,3)

In [3]:
W[2]

array([6, 7, 8])

In [4]:
W[5]

array([15, 16, 17])

In [5]:
idx = np.array([1,0,3,0])

In [6]:
W[idx] #여러 개 반환 가능

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [7]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    # 가중치 기울기 dW를 꺼낸 다음 dW의 원소를 0으로 덮어쓴다.
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        # dW[self.idx] = dout # 앞 층에서 전해진 기울기 dout를 할당; 나쁜 예
        
        '''
        idx가 중복된 원소를 가지고 있다면 dh를 해당 행에 할당할 때 문제가 생긴다.
        '''
        np.add.at(dW, self.idx, dout)
        return None

## 4.2 개선 2 - 네거티브 샘플링

행렬 곱과 softmax 계층의 계산 병목을 해소하는 목적  
softmax 대신 네거티브 샘플링을 이용하면 어휘가 아무리 많아져도 계산량을 낮은 수준에서 일정하게 억제 가능하다.

### 은닉충 이후 계산의 문제점

어휘가 100만개일 때, Embedding 계층을 도입하려 입력층 계산에서의 낭비를 줄였다. 남은 문제는 은닉충 이후의 처리. 
1. 은닉충의 뉴런과 가중치 행렬  $W_{out}$ 의 곱 - 큰 행렬의 곱을 계산하려면 시간도 오래 걸리고 메모리도 많이 필요하다
2. softmax 계층의 계산 - 어휘가 많아지면 softmax의 계산량도 증가

위 두 부분에서 계산이 오래 걸린다

### 다중분류 --> 이진분류

핵심 아이디어: 이진 분류/ 이진 분류로 근사하는 것
==> 이렇게 하면 뉴런을 하나만준비하면 된다.   
은닉충과 출력 층의 가중치 행렬의 내적은 특정 단어에 해당하는 열(단어 벡터) 만을 추출하고, 그 추출된 벡터와 은닉충 뉴런과의 내적을 계산하면 끝이다. 이 계산 값이 최종 점수.

이전까지의 출력층에서 모든 단어를 대상으로 계산을 수행한거라면 여기에서는 특정 단어 하나에 주목하여 그 점수만을 계산하는 것. 그리고 시그모이드 함수를 이용해 그 점수를 확률로 변환

### 시그모이드 함수와 교차 엔트로피 오차

이진분류의 손실을 구할 때는 손실 함수로 '교차 엔트로피 오차'를 사용 

교차 엔트로피 식

오차가 크면 크게 학습하고 오차가 작으면 작게 학습한다.

### 구현

In [8]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections


class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh


### 네거티브 샘플링

#### 네거티브 샘플링 구현

#### 네거티브 샘플링 기법

## 4.3 개선판 word2vec 학습

### CBOW 모델 
개선된 simpleCBOW 의 모델: Embedding 계층와 Negative Samplint Loss 계층을 적용하는 것. 

In [9]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    #초기화 4가지 인수: 어휘 수, 은닉충의 뉴런 수, 단어 ID의 목록
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


### CBOW 모델 학습 코드

In [10]:
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from ch04.cbow import CBOW
from ch04.skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


| 에폭 1 |  반복 1 / 5619 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 5619 | 시간 5[s] | 손실 4.16
| 에폭 1 |  반복 41 / 5619 | 시간 9[s] | 손실 4.15
| 에폭 1 |  반복 61 / 5619 | 시간 14[s] | 손실 4.12
| 에폭 1 |  반복 81 / 5619 | 시간 18[s] | 손실 4.05
| 에폭 1 |  반복 101 / 5619 | 시간 23[s] | 손실 3.94
| 에폭 1 |  반복 121 / 5619 | 시간 28[s] | 손실 3.78
| 에폭 1 |  반복 141 / 5619 | 시간 33[s] | 손실 3.64
| 에폭 1 |  반복 161 / 5619 | 시간 37[s] | 손실 3.49
| 에폭 1 |  반복 181 / 5619 | 시간 42[s] | 손실 3.37
| 에폭 1 |  반복 201 / 5619 | 시간 47[s] | 손실 3.24
| 에폭 1 |  반복 221 / 5619 | 시간 51[s] | 손실 3.16
| 에폭 1 |  반복 241 / 5619 | 시간 56[s] | 손실 3.08
| 에폭 1 |  반복 261 / 5619 | 시간 61[s] | 손실 3.01
| 에폭 1 |  반복 281 / 5619 | 시간 65[s] | 손실 2.98
| 에폭 1 |  반복 301 / 5619 | 시간 70[s] | 손실 2.89
| 에폭 1 |  반복 321 / 5619 | 시간 75[s] | 손실 2.88
| 에폭 1 |  반복 341 / 5619 | 시간 81[s] | 손실 2.85
| 에폭 1 |  반복 361 / 5619 | 시간 88[s] | 손실 2.81
| 에폭 1 |  반복 381 / 5619 | 시간 94[s] | 손실 2.76
| 에폭 1 |  반복 401 / 5619 | 시간 100[s] | 손실 2.75
| 에폭 1 |  반복 421 / 5619 | 시간 106[s] | 손실 2.73
| 에폭 1 |  반복 441 

| 에폭 1 |  반복 3521 / 5619 | 시간 1155[s] | 손실 2.40
| 에폭 1 |  반복 3541 / 5619 | 시간 1160[s] | 손실 2.42
| 에폭 1 |  반복 3561 / 5619 | 시간 1164[s] | 손실 2.42
| 에폭 1 |  반복 3581 / 5619 | 시간 1169[s] | 손실 2.43
| 에폭 1 |  반복 3601 / 5619 | 시간 1173[s] | 손실 2.39
| 에폭 1 |  반복 3621 / 5619 | 시간 1177[s] | 손실 2.44
| 에폭 1 |  반복 3641 / 5619 | 시간 1181[s] | 손실 2.41
| 에폭 1 |  반복 3661 / 5619 | 시간 1185[s] | 손실 2.43
| 에폭 1 |  반복 3681 / 5619 | 시간 1189[s] | 손실 2.41
| 에폭 1 |  반복 3701 / 5619 | 시간 1194[s] | 손실 2.45
| 에폭 1 |  반복 3721 / 5619 | 시간 1198[s] | 손실 2.38
| 에폭 1 |  반복 3741 / 5619 | 시간 1203[s] | 손실 2.42
| 에폭 1 |  반복 3761 / 5619 | 시간 1207[s] | 손실 2.40
| 에폭 1 |  반복 3781 / 5619 | 시간 1212[s] | 손실 2.39
| 에폭 1 |  반복 3801 / 5619 | 시간 1217[s] | 손실 2.37
| 에폭 1 |  반복 3821 / 5619 | 시간 1222[s] | 손실 2.40
| 에폭 1 |  반복 3841 / 5619 | 시간 1227[s] | 손실 2.40
| 에폭 1 |  반복 3861 / 5619 | 시간 1232[s] | 손실 2.40
| 에폭 1 |  반복 3881 / 5619 | 시간 1236[s] | 손실 2.42
| 에폭 1 |  반복 3901 / 5619 | 시간 1240[s] | 손실 2.36
| 에폭 1 |  반복 3921 / 5619 | 시간 1243[s] | 

| 에폭 2 |  반복 1341 / 5619 | 시간 1891[s] | 손실 2.28
| 에폭 2 |  반복 1361 / 5619 | 시간 1895[s] | 손실 2.29
| 에폭 2 |  반복 1381 / 5619 | 시간 1900[s] | 손실 2.27
| 에폭 2 |  반복 1401 / 5619 | 시간 1904[s] | 손실 2.27
| 에폭 2 |  반복 1421 / 5619 | 시간 1908[s] | 손실 2.27
| 에폭 2 |  반복 1441 / 5619 | 시간 1912[s] | 손실 2.27
| 에폭 2 |  반복 1461 / 5619 | 시간 1917[s] | 손실 2.26
| 에폭 2 |  반복 1481 / 5619 | 시간 1921[s] | 손실 2.27
| 에폭 2 |  반복 1501 / 5619 | 시간 1925[s] | 손실 2.28
| 에폭 2 |  반복 1521 / 5619 | 시간 1929[s] | 손실 2.27
| 에폭 2 |  반복 1541 / 5619 | 시간 1934[s] | 손실 2.27
| 에폭 2 |  반복 1561 / 5619 | 시간 1938[s] | 손실 2.28
| 에폭 2 |  반복 1581 / 5619 | 시간 1942[s] | 손실 2.28
| 에폭 2 |  반복 1601 / 5619 | 시간 1947[s] | 손실 2.24
| 에폭 2 |  반복 1621 / 5619 | 시간 1951[s] | 손실 2.26
| 에폭 2 |  반복 1641 / 5619 | 시간 1955[s] | 손실 2.29
| 에폭 2 |  반복 1661 / 5619 | 시간 1960[s] | 손실 2.28
| 에폭 2 |  반복 1681 / 5619 | 시간 1964[s] | 손실 2.30
| 에폭 2 |  반복 1701 / 5619 | 시간 1968[s] | 손실 2.25
| 에폭 2 |  반복 1721 / 5619 | 시간 1972[s] | 손실 2.24
| 에폭 2 |  반복 1741 / 5619 | 시간 1977[s] | 

| 에폭 2 |  반복 4761 / 5619 | 시간 2627[s] | 손실 2.15
| 에폭 2 |  반복 4781 / 5619 | 시간 2631[s] | 손실 2.18
| 에폭 2 |  반복 4801 / 5619 | 시간 2636[s] | 손실 2.12
| 에폭 2 |  반복 4821 / 5619 | 시간 2640[s] | 손실 2.17
| 에폭 2 |  반복 4841 / 5619 | 시간 2644[s] | 손실 2.16
| 에폭 2 |  반복 4861 / 5619 | 시간 2648[s] | 손실 2.18
| 에폭 2 |  반복 4881 / 5619 | 시간 2652[s] | 손실 2.16
| 에폭 2 |  반복 4901 / 5619 | 시간 2657[s] | 손실 2.15
| 에폭 2 |  반복 4921 / 5619 | 시간 2661[s] | 손실 2.14
| 에폭 2 |  반복 4941 / 5619 | 시간 2665[s] | 손실 2.16
| 에폭 2 |  반복 4961 / 5619 | 시간 2670[s] | 손실 2.13
| 에폭 2 |  반복 4981 / 5619 | 시간 2674[s] | 손실 2.13
| 에폭 2 |  반복 5001 / 5619 | 시간 2679[s] | 손실 2.15
| 에폭 2 |  반복 5021 / 5619 | 시간 2683[s] | 손실 2.16
| 에폭 2 |  반복 5041 / 5619 | 시간 2687[s] | 손실 2.16
| 에폭 2 |  반복 5061 / 5619 | 시간 2691[s] | 손실 2.16
| 에폭 2 |  반복 5081 / 5619 | 시간 2695[s] | 손실 2.17
| 에폭 2 |  반복 5101 / 5619 | 시간 2700[s] | 손실 2.17
| 에폭 2 |  반복 5121 / 5619 | 시간 2704[s] | 손실 2.15
| 에폭 2 |  반복 5141 / 5619 | 시간 2708[s] | 손실 2.17
| 에폭 2 |  반복 5161 / 5619 | 시간 2713[s] | 

| 에폭 3 |  반복 2581 / 5619 | 시간 3554[s] | 손실 2.05
| 에폭 3 |  반복 2601 / 5619 | 시간 3557[s] | 손실 2.05
| 에폭 3 |  반복 2621 / 5619 | 시간 3561[s] | 손실 2.05
| 에폭 3 |  반복 2641 / 5619 | 시간 3565[s] | 손실 2.04
| 에폭 3 |  반복 2661 / 5619 | 시간 3568[s] | 손실 2.04
| 에폭 3 |  반복 2681 / 5619 | 시간 3572[s] | 손실 2.04
| 에폭 3 |  반복 2701 / 5619 | 시간 3575[s] | 손실 2.08
| 에폭 3 |  반복 2721 / 5619 | 시간 3579[s] | 손실 2.05
| 에폭 3 |  반복 2741 / 5619 | 시간 3583[s] | 손실 2.08
| 에폭 3 |  반복 2761 / 5619 | 시간 3587[s] | 손실 2.02
| 에폭 3 |  반복 2781 / 5619 | 시간 3591[s] | 손실 2.04
| 에폭 3 |  반복 2801 / 5619 | 시간 3595[s] | 손실 2.06
| 에폭 3 |  반복 2821 / 5619 | 시간 3599[s] | 손실 2.05
| 에폭 3 |  반복 2841 / 5619 | 시간 3603[s] | 손실 2.05
| 에폭 3 |  반복 2861 / 5619 | 시간 3606[s] | 손실 2.08
| 에폭 3 |  반복 2881 / 5619 | 시간 3610[s] | 손실 2.02
| 에폭 3 |  반복 2901 / 5619 | 시간 3613[s] | 손실 2.00
| 에폭 3 |  반복 2921 / 5619 | 시간 3617[s] | 손실 2.03
| 에폭 3 |  반복 2941 / 5619 | 시간 3620[s] | 손실 2.03
| 에폭 3 |  반복 2961 / 5619 | 시간 3624[s] | 손실 2.03
| 에폭 3 |  반복 2981 / 5619 | 시간 3627[s] | 

...

| 에폭 6 |  반복 2901 / 5619 | 시간 6842[s] | 손실 1.74
| 에폭 6 |  반복 2921 / 5619 | 시간 6845[s] | 손실 1.73
| 에폭 6 |  반복 2941 / 5619 | 시간 6849[s] | 손실 1.77
| 에폭 6 |  반복 2961 / 5619 | 시간 6852[s] | 손실 1.70
| 에폭 6 |  반복 2981 / 5619 | 시간 6856[s] | 손실 1.76
| 에폭 6 |  반복 3001 / 5619 | 시간 6860[s] | 손실 1.74
| 에폭 6 |  반복 3021 / 5619 | 시간 6863[s] | 손실 1.77
| 에폭 6 |  반복 3041 / 5619 | 시간 6867[s] | 손실 1.77
| 에폭 6 |  반복 3061 / 5619 | 시간 6870[s] | 손실 1.76
| 에폭 6 |  반복 3081 / 5619 | 시간 6873[s] | 손실 1.71
| 에폭 6 |  반복 3101 / 5619 | 시간 6877[s] | 손실 1.77
| 에폭 6 |  반복 3121 / 5619 | 시간 6880[s] | 손실 1.70
| 에폭 6 |  반복 3141 / 5619 | 시간 6884[s] | 손실 1.74
| 에폭 6 |  반복 3161 / 5619 | 시간 6887[s] | 손실 1.71
| 에폭 6 |  반복 3181 / 5619 | 시간 6890[s] | 손실 1.74
| 에폭 6 |  반복 3201 / 5619 | 시간 6894[s] | 손실 1.74
| 에폭 6 |  반복 3221 / 5619 | 시간 6897[s] | 손실 1.78
| 에폭 6 |  반복 3241 / 5619 | 시간 6900[s] | 손실 1.70
| 에폭 6 |  반복 3261 / 5619 | 시간 6904[s] | 손실 1.71
| 에폭 6 |  반복 3281 / 5619 | 시간 6907[s] | 손실 1.74
| 에폭 6 |  반복 3301 / 5619 | 시간 6910[s] | 

| 에폭 7 |  반복 721 / 5619 | 시간 7482[s] | 손실 1.67
| 에폭 7 |  반복 741 / 5619 | 시간 7487[s] | 손실 1.68
| 에폭 7 |  반복 761 / 5619 | 시간 7491[s] | 손실 1.69
| 에폭 7 |  반복 781 / 5619 | 시간 7496[s] | 손실 1.65
| 에폭 7 |  반복 801 / 5619 | 시간 7501[s] | 손실 1.63
| 에폭 7 |  반복 821 / 5619 | 시간 7505[s] | 손실 1.68
| 에폭 7 |  반복 841 / 5619 | 시간 7510[s] | 손실 1.71
| 에폭 7 |  반복 861 / 5619 | 시간 7514[s] | 손실 1.66
| 에폭 7 |  반복 881 / 5619 | 시간 7519[s] | 손실 1.70
| 에폭 7 |  반복 901 / 5619 | 시간 7523[s] | 손실 1.65
| 에폭 7 |  반복 921 / 5619 | 시간 7528[s] | 손실 1.67
| 에폭 7 |  반복 941 / 5619 | 시간 7533[s] | 손실 1.67
| 에폭 7 |  반복 961 / 5619 | 시간 7537[s] | 손실 1.70
| 에폭 7 |  반복 981 / 5619 | 시간 7542[s] | 손실 1.68
| 에폭 7 |  반복 1001 / 5619 | 시간 7546[s] | 손실 1.71
| 에폭 7 |  반복 1021 / 5619 | 시간 7552[s] | 손실 1.67
| 에폭 7 |  반복 1041 / 5619 | 시간 7558[s] | 손실 1.63
| 에폭 7 |  반복 1061 / 5619 | 시간 7562[s] | 손실 1.70
| 에폭 7 |  반복 1081 / 5619 | 시간 7567[s] | 손실 1.66
| 에폭 7 |  반복 1101 / 5619 | 시간 7571[s] | 손실 1.71
| 에폭 7 |  반복 1121 / 5619 | 시간 7576[s] | 손실 1.70
| 에폭 7

| 에폭 7 |  반복 4141 / 5619 | 시간 8396[s] | 손실 1.67
| 에폭 7 |  반복 4161 / 5619 | 시간 8405[s] | 손실 1.65
| 에폭 7 |  반복 4181 / 5619 | 시간 8411[s] | 손실 1.70
| 에폭 7 |  반복 4201 / 5619 | 시간 8417[s] | 손실 1.64
| 에폭 7 |  반복 4221 / 5619 | 시간 8424[s] | 손실 1.70
| 에폭 7 |  반복 4241 / 5619 | 시간 8430[s] | 손실 1.68
| 에폭 7 |  반복 4261 / 5619 | 시간 8436[s] | 손실 1.67
| 에폭 7 |  반복 4281 / 5619 | 시간 8442[s] | 손실 1.66
| 에폭 7 |  반복 4301 / 5619 | 시간 8449[s] | 손실 1.69
| 에폭 7 |  반복 4321 / 5619 | 시간 8455[s] | 손실 1.69
| 에폭 7 |  반복 4341 / 5619 | 시간 8462[s] | 손실 1.64
| 에폭 7 |  반복 4361 / 5619 | 시간 8469[s] | 손실 1.70
| 에폭 7 |  반복 4381 / 5619 | 시간 8476[s] | 손실 1.68
| 에폭 7 |  반복 4401 / 5619 | 시간 8482[s] | 손실 1.67
| 에폭 7 |  반복 4421 / 5619 | 시간 8489[s] | 손실 1.67
| 에폭 7 |  반복 4441 / 5619 | 시간 8496[s] | 손실 1.69
| 에폭 7 |  반복 4461 / 5619 | 시간 8504[s] | 손실 1.65
| 에폭 7 |  반복 4481 / 5619 | 시간 8511[s] | 손실 1.66
| 에폭 7 |  반복 4501 / 5619 | 시간 8518[s] | 손실 1.68
| 에폭 7 |  반복 4521 / 5619 | 시간 8526[s] | 손실 1.66
| 에폭 7 |  반복 4541 / 5619 | 시간 8534[s] | 

| 에폭 8 |  반복 1961 / 5619 | 시간 9721[s] | 손실 1.62
| 에폭 8 |  반복 1981 / 5619 | 시간 9731[s] | 손실 1.60
| 에폭 8 |  반복 2001 / 5619 | 시간 9740[s] | 손실 1.62
| 에폭 8 |  반복 2021 / 5619 | 시간 9750[s] | 손실 1.55
| 에폭 8 |  반복 2041 / 5619 | 시간 9759[s] | 손실 1.63
| 에폭 8 |  반복 2061 / 5619 | 시간 9769[s] | 손실 1.63
| 에폭 8 |  반복 2081 / 5619 | 시간 9781[s] | 손실 1.63
| 에폭 8 |  반복 2101 / 5619 | 시간 9791[s] | 손실 1.60
| 에폭 8 |  반복 2121 / 5619 | 시간 9801[s] | 손실 1.63
| 에폭 8 |  반복 2141 / 5619 | 시간 9811[s] | 손실 1.61
| 에폭 8 |  반복 2161 / 5619 | 시간 9822[s] | 손실 1.60
| 에폭 8 |  반복 2181 / 5619 | 시간 9832[s] | 손실 1.58
| 에폭 8 |  반복 2201 / 5619 | 시간 9841[s] | 손실 1.61
| 에폭 8 |  반복 2221 / 5619 | 시간 9850[s] | 손실 1.60
| 에폭 8 |  반복 2241 / 5619 | 시간 9861[s] | 손실 1.56
| 에폭 8 |  반복 2261 / 5619 | 시간 9868[s] | 손실 1.60
| 에폭 8 |  반복 2281 / 5619 | 시간 9875[s] | 손실 1.57
| 에폭 8 |  반복 2301 / 5619 | 시간 9881[s] | 손실 1.61
| 에폭 8 |  반복 2321 / 5619 | 시간 9888[s] | 손실 1.57
| 에폭 8 |  반복 2341 / 5619 | 시간 9894[s] | 손실 1.61
| 에폭 8 |  반복 2361 / 5619 | 시간 9901[s] | 

KeyboardInterrupt: 

### CBOW 모델 평가

In [11]:
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)

FileNotFoundError: [Errno 2] No such file or directory: 'cbow_params.pkl'

word2vec 으로 얻은 단어의 분산 표현은 비슷한 단어를 가까이 모을 뿐 아니라, 더 복잡한 패턴을 파악하는 것으로 알려져 있다. 
word2vec의 단어의 분산 표현을 사용하면 유추 문제를 벡터의 덧셈과 뺼셈으로 풀 수 있다. 

## 4.4 word2vec 남은 주제

word2vec으로 얻은 단어의 분산 표현은 비슷한 단어를 찾는 용도로 이용할 수 있다. 
1.  전이 학습 -  한 분야에서 배운 지식을 다른 분야에도 적용하는 기법이다.  

먼저 큰 말뭉치로 학습을 한 후, 그 분산 표현을 각자의 작업에 이용할 수 있다. 

2. 단어를 고정 길이 벡터로 변환해준다
자연어로 쓰여진 질문을 고정 길이 벡터로 변환할 수 있다면, 그 벡터를 다른 머신러닝 시스템의 입력으로 사용 가능하다.

## 4.5 정리

**CBOW 모델 개선**
1. Embedding 계층 구현
2. 네거티브 샘플링 기법 도입

but, 말뭉치의 어휘 수 증가에 비례하여 계산량이 증가하는 문제  



네거티브 샘플링: '모두' 대신 '일부'를 처리하는 것. 일부 단어만을 대상으로 하여 계산을 효율적으로 수행한다.

- Embedding 계층은 단어의 분산 표현을 담고 있으며, 순전파 시 지정한 단어 ID 벡터를 추출
- word2vec은 어휘 수의 증가에 비례하여 계산량도 증가하므로, 근사치로 계산하는 빠른 방법을 사용하면 좋다
- 네거티브 샘플링은 부정적 예를 몇 개 샘플링 하는 기법. 다중분류를 이진 분류처럼 취급할 수 있다.
- word2vec으로 얻은 단어의 분산 표현에는 단어의 의미가 있다. 비슷한 맥락에서 사용되는 단어는 단어의 벡터 공간에 가까이 위치한다
- word2vec의 단어의 분산 표현을 이용하면 유추 문제를 벡터의 덧셈과 뺄셈으로 풀 수 있게 된다
- wore2vec 은 전이 학습 측면에서 중요. 단어의 분산 표현은 다양한 자연어 처리 작업에 이용 할 수 있다.